In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357292 sha256=149a0847de48c3c097f4d46bfd0a3e3cf548fd389ec098844a96064defb8c3d4
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from surprise import Dataset, Reader, KNNWithMeans,SVD, SVDpp,NMF, BaselineOnly, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from collections import defaultdict

In [ ]:
movies        = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/tags.csv')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
movies.isnull().sum()

,0
movieId,0
title,0
genres,0


In [ ]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [ ]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [ ]:
links.isnull().sum() # There is some missing values in links dataframe

,0
movieId,0
imdbId,0
tmdbId,8


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


In [ ]:
ratings['userId'].nunique()

610

In [ ]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [ ]:
tags.isnull().sum()

,0
userId,0
movieId,0
tag,0
timestamp,0


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

# Movie Recommendation : KNNWithMeans

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.20)

In [ ]:
# Initialize and train the KNNWithMeans model
model = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True})
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"KNNWithMeans training completed in {fit_time:.2f} seconds.")

Computing the cosine similarity matrix...
Done computing similarity matrix.
KNNWithMeans training completed in 0.40 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)
predictions

[Prediction(uid=40, iid=432, r_ui=3.0, est=2.9690861489568237, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=414, iid=348, r_ui=4.0, est=3.5203672153694203, details={'actual_k': 26, 'was_impossible': False}),
 Prediction(uid=288, iid=1968, r_ui=3.0, est=3.317638601018522, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=610, iid=6214, r_ui=4.5, est=3.921671410958643, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=432, iid=84189, r_ui=3.5, est=3.40609756097561, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=249, iid=52458, r_ui=4.0, est=3.4980512603794884, details={'actual_k': 7, 'was_impossible': False}),
 Prediction(uid=307, iid=48520, r_ui=3.0, est=2.200309166397889, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid=600, iid=4310, r_ui=2.0, est=2.534820770530376, details={'actual_k': 31, 'was_impossible': False}),
 Prediction(uid=414, iid=2044, r_ui=2.0, est=3.776529522760695, details={'

In [ ]:
# Calculate MSE
mse = accuracy.mse(predictions, verbose=False)

# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = accuracy.mae(predictions, verbose=False)

# Print the results
print(f"Training Time: {fit_time:.2f} seconds")
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")

Training Time: 0.40 seconds
Test MSE: 0.8193
Test RMSE: 0.9052
Test MAE: 0.6927


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some Movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some Movies recommendations for User 1:
Movie ID: 47, Estimated Rating: 4.74
Movie ID: 1080, Estimated Rating: 4.73
Movie ID: 1136, Estimated Rating: 4.72
Movie ID: 1291, Estimated Rating: 4.71
Movie ID: 943, Estimated Rating: 4.70
Movie ID: 1210, Estimated Rating: 4.70
Movie ID: 457, Estimated Rating: 4.67
Movie ID: 940, Estimated Rating: 4.67
Movie ID: 4006, Estimated Rating: 4.64
Movie ID: 2949, Estimated Rating: 4.63


In [ ]:
from collections import defaultdict

# Function to count predictions for each user ID
def count_predictions(predictions):
    prediction_counts = defaultdict(int)
    for prediction in predictions:
        prediction_counts[prediction.uid] += 1
    return prediction_counts

# Generate predictions if not already done
# Ensure you have a predictions list from model.test(testset) or similar

# Count predictions for each user
prediction_counts = count_predictions(predictions)

# Print the number of predictions for each user ID
for user_id, count in prediction_counts.items():
    print(f"User ID {user_id} has {count} predictions.")

# You might want to print the counts for specific users, such as user 1 or 2
print(f"Number of predictions for user 1: {prediction_counts[1]}")
if 2 in prediction_counts:
    print(f"Number of predictions for user 2: {prediction_counts[2]}")
else:
    print("No predictions found for user 2.")


User ID 40 has 23 predictions.
User ID 414 has 524 predictions.
User ID 288 has 220 predictions.
User ID 610 has 246 predictions.
User ID 432 has 55 predictions.
User ID 249 has 232 predictions.
User ID 307 has 185 predictions.
User ID 600 has 146 predictions.
User ID 533 has 6 predictions.
User ID 298 has 182 predictions.
User ID 280 has 30 predictions.
User ID 294 has 91 predictions.
User ID 430 has 8 predictions.
User ID 497 has 9 predictions.
User ID 32 has 19 predictions.
User ID 387 has 202 predictions.
User ID 125 has 77 predictions.
User ID 445 has 18 predictions.
User ID 453 has 55 predictions.
User ID 304 has 41 predictions.
User ID 517 has 88 predictions.
User ID 232 has 174 predictions.
User ID 64 has 114 predictions.
User ID 608 has 170 predictions.
User ID 318 has 185 predictions.
User ID 45 has 81 predictions.
User ID 555 has 99 predictions.
User ID 495 has 43 predictions.
User ID 606 has 214 predictions.
User ID 599 has 517 predictions.
User ID 105 has 154 predictions.


# SVD

In [ ]:
# Initialize and train the SVD model
model = SVD()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Evaluate MSE, RMSE, and MAE
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
mae = accuracy.mae(predictions, verbose=False)

# Print results in four lines
print(f"KNNWithMeans training completed in {fit_time:.2f} seconds.")
print(f"KNNWithMeans: MSE={mse:.3f}")
print(f"KNNWithMeans: RMSE={rmse:.3f}")
print(f"KNNWithMeans: MAE={mae:.3f}")


KNNWithMeans training completed in 1.15 seconds.
KNNWithMeans: MSE=0.771
KNNWithMeans: RMSE=0.878
KNNWithMeans: MAE=0.673


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some movies recommendations for User 1:
Movie ID: 1270, Estimated Rating: 4.99
Movie ID: 1210, Estimated Rating: 4.91
Movie ID: 1291, Estimated Rating: 4.75
Movie ID: 1136, Estimated Rating: 4.66
Movie ID: 223, Estimated Rating: 4.61
Movie ID: 1080, Estimated Rating: 4.59
Movie ID: 457, Estimated Rating: 4.56
Movie ID: 1032, Estimated Rating: 4.55
Movie ID: 1, Estimated Rating: 4.55
Movie ID: 2916, Estimated Rating: 4.54


# SVD++

In [ ]:
# Initialize and train the SVD++ model
model = SVDpp()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"SVD++ training completed in {fit_time:.2f} seconds.")

SVD++ training completed in 83.91 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Evaluate MSE, RMSE, and MAE
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
mae = accuracy.mae(predictions, verbose=False)

# Print results
print(f"SVD++ training completed in {fit_time:.2f} seconds.")
print(f"SVD++: MSE={mse:.3f}")
print(f"SVD++: RMSE={rmse:.3f}")
print(f"SVD++: MAE={mae:.3f}")

SVD++ training completed in 83.91 seconds.
SVD++: MSE=0.749
SVD++: RMSE=0.865
SVD++: MAE=0.662


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Top 10 recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Top 10 recommendations for User 1:
Movie ID: 1080, Estimated Rating: 4.80
Movie ID: 457, Estimated Rating: 4.78
Movie ID: 223, Estimated Rating: 4.75
Movie ID: 1136, Estimated Rating: 4.72
Movie ID: 1291, Estimated Rating: 4.68
Movie ID: 47, Estimated Rating: 4.67
Movie ID: 1, Estimated Rating: 4.60
Movie ID: 2174, Estimated Rating: 4.57
Movie ID: 1270, Estimated Rating: 4.57
Movie ID: 151, Estimated Rating: 4.56


# NMF

In [ ]:
# Initialize and train the NMF model
model = NMF()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"NMF training completed in {fit_time:.2f} seconds.")

NMF training completed in 2.10 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Evaluate MSE, RMSE, and MAE
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
mae = accuracy.mae(predictions, verbose=False)

# Print results
print(f"NMF training completed in {fit_time:.2f} seconds.")
print(f"NMF: MSE={mse:.3f}")
print(f"NMF: RMSE={rmse:.3f}")
print(f"NMF: MAE={mae:.3f}")


NMF training completed in 2.10 seconds.
NMF: MSE=0.857
NMF: RMSE=0.926
NMF: MAE=0.708


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some movies recommendations for User 1:
Movie ID: 1080, Estimated Rating: 5.00
Movie ID: 1210, Estimated Rating: 5.00
Movie ID: 1136, Estimated Rating: 5.00
Movie ID: 47, Estimated Rating: 4.93
Movie ID: 223, Estimated Rating: 4.87
Movie ID: 2115, Estimated Rating: 4.85
Movie ID: 2427, Estimated Rating: 4.80
Movie ID: 1270, Estimated Rating: 4.79
Movie ID: 1291, Estimated Rating: 4.77
Movie ID: 3034, Estimated Rating: 4.74


# Baselineonly

In [ ]:
# Initialize and train the BaselineOnly model
bsl_options = {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}
model = BaselineOnly(bsl_options=bsl_options)
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"BaselineOnly training completed in {fit_time:.2f} seconds.")

Estimating biases using als...
BaselineOnly training completed in 0.07 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Evaluate MSE, RMSE, and MAE
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
mae = accuracy.mae(predictions, verbose=False)

# Print results
print(f"BaselineOnly training completed in {fit_time:.2f} seconds.")
print(f"BaselineOnly: MSE={mse:.3f}")
print(f"BaselineOnly: RMSE={rmse:.3f}")
print(f"BaselineOnly: MAE={mae:.3f}")


BaselineOnly training completed in 0.07 seconds.
BaselineOnly: MSE=0.759
BaselineOnly: RMSE=0.871
BaselineOnly: MAE=0.670


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some Movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some Movies recommendations for User 1:
Movie ID: 1136, Estimated Rating: 4.89
Movie ID: 1210, Estimated Rating: 4.81
Movie ID: 1291, Estimated Rating: 4.76
Movie ID: 1270, Estimated Rating: 4.73
Movie ID: 457, Estimated Rating: 4.70
Movie ID: 47, Estimated Rating: 4.69
Movie ID: 1080, Estimated Rating: 4.67
Movie ID: 223, Estimated Rating: 4.62
Movie ID: 1, Estimated Rating: 4.62
Movie ID: 3578, Estimated Rating: 4.59
